# Algoritmo genético

Una empresa de paquetería quiere repartir en una ciudad un conjunto de paquetes en distintas 
casas. Para ello quiere saber cual es la ruta más corta para repartir todos los paquetes. Un 
paquete tiene un peso y una prioridad de entrega (generados aleatoriamente) y un repartidor 
tiene una capacidad máxima de cuantos paquetes puede llevar al mismo tiempo, basado en 
el peso (generado aleatoriamente). <br>

Dado un grafo grande de casas (definido por el mismo equipo), donde los nodos representan 
una casa a donde se puede entregar un paquete, y existe una distancia entre cada nodo. 
Cuando el repartidor se quede sin paquetes que entregar debe regresar a la matriz de la 
empresa para verificar si existen paquetes por entregar y así hasta quedarse sin paquetes. El 
objetivo es empezar en el nodo de la empresa y minimizar el costo total de las rutas para 
entregar eficientemente los paquetes en el menor tiempo y con la menor distancia recorrida. 

Posibles datos de entrada: <br>
• Conjunto de paquetes con peso, prioridad y nodo de entrega.<br>
• Vehículo de entregas con peso máximo de paquetes que puede llevar.<br>
• Mapa de rutas que contiene el grafo de información.<br>

Objetivos:<br>
• Minimizar la distancia total recorrida por los vehículos.<br>
• Minimizar los tiempos de entrega de los paquetes.<br>
• Entregar todos los paquetes.<br>

Restricciones:<br>
• Los vehículos no deben superar su capacidad máxima<br>
• Todos los paquetes deben ser entregados<br>
• Los nodos del grafo no están todos interconectados, puedes necesitar ir a otra casa 
antes de llegar al objetivo.<br>

Sugerencias adicionales:<br>
• Empezar con un algoritmo genético aplicado a encontrar una ruta eficiente en un grafo
de ida y vuelta a varios puntos, y luego agregar las restricciones de paquetes y peso.<br>
• Probar diferentes configuraciones de parámetros para la configuración del genotipo, la 
evaluación de la solución, la generación de una nueva población o el factor de 
mutación.

In [2]:
import random
import heapq
import numpy as np

# Grafo ciudades
grafo = {
    "Arad": {"Zerind": 36.25, "Sibiu": 100.96, "Timisoara": 68.07},
    "Zerind": {"Oradea": 37.59, "Arad": 36.25},
    "Oradea": {"Sibiu": 113.72, "Zerind": 37.59},
    "Timisoara": {"Lugoj": 64.47, "Arad": 68.07},
    "Lugoj": {"Mehadia": 33.06, "Timisoara": 64.47},
    "Mehadia": {"Drobeta": 33.06, "Lugoj": 33.06},
    "Drobeta": {"Craiova": 72.69, "Mehadia": 33.06},
    "Sibiu": {"Fagaras": 82.3, "Rimnicu Vilcea": 45.18, "Oradea": 113.72, "Arad": 100.96},
    "Fagaras": {"Bucharest": 118.34, "Sibiu": 82.3},
    "Rimnicu Vilcea": {"Pitesti": 81.39, "Craiova": 103.25, "Sibiu": 45.18},
    "Craiova": {"Pitesti": 87.21, "Drobeta": 72.69},
    "Pitesti": {"Bucharest": 58.25, "Rimnicu Vilcea": 81.39},
    "Bucharest": {"Giurgiu": 47.17, "Urziceni": 67.72, "Fagaras": 118.34}, # <-- Empresa
    "Giurgiu": {"Bucharest": 47.17},
    "Urziceni": {"Hirsova": 63.0, "Vaslui": 89.94, "Bucharest": 67.72},
    "Hirsova": {"Eforie": 51.88, "Urziceni": 63.0},
    "Eforie": {"Hirsova": 51.88},
    "Vaslui": {"Iasi": 58.67, "Urziceni": 89.94},
    "Iasi": {"Neamt": 62.24, "Vaslui": 58.67},
    "Neamt": {"Iasi": 62.24}
}

# Generar paquetes de ejemplo
def generar_paquetes(num_paquetes, nodos):
    return [
        {
            'peso': random.randint(1, 10),
            'prioridad': random.randint(1, 10),
            'nodo_entrega': random.choice(nodos),
        }
        for _ in range(num_paquetes)
    ]

# Implementación de A*
def a_star(grafo, inicio, objetivo):
    cola_prioridad = []
    heapq.heappush(cola_prioridad, (0, inicio))
    distancias = {nodo: float('inf') for nodo in grafo}
    distancias[inicio] = 0
    caminos = {nodo: None for nodo in grafo}

    while cola_prioridad:
        _, nodo_actual = heapq.heappop(cola_prioridad)
        if nodo_actual == objetivo:
            break
        for vecino, peso in grafo[nodo_actual].items():
            nueva_distancia = distancias[nodo_actual] + peso
            if nueva_distancia < distancias[vecino]:
                distancias[vecino] = nueva_distancia
                caminos[vecino] = nodo_actual
                heapq.heappush(cola_prioridad, (nueva_distancia, vecino))

    camino = []
    nodo = objetivo
    while nodo:
        camino.append(nodo)
        nodo = caminos[nodo]
    return camino[::-1], distancias[objetivo]

# Evaluar el costo total usando A*
def calcular_costo_total(individuo, grafo, capacidad, alpha=10):
    distancia_total = 0
    peso_actual = 0
    nodo_actual = 'Bucharest'
    suma_prioridades = 0

    print(f"\nEvaluando individuo con carga: {individuo['carga']}")

    for paquete in individuo['carga']:
        if peso_actual + paquete['peso'] > capacidad:
            # Regresar a la empresa
            camino, distancia = a_star(grafo, nodo_actual, 'Bucharest')
            print(f"Regresando a la Empresa desde {nodo_actual} (Ruta: {camino})")
            distancia_total += distancia
            nodo_actual = 'Bucharest'
            peso_actual = 0

        # Entregar el paquete
        camino, distancia = a_star(grafo, nodo_actual, paquete['nodo_entrega'])
        print(f"Entregando paquete a {paquete['nodo_entrega']} (Ruta: {camino})")
        distancia_total += distancia
        nodo_actual = paquete['nodo_entrega']
        peso_actual += paquete['peso']
        suma_prioridades += paquete['prioridad']

    # Regresar a la empresa al final
    camino, distancia = a_star(grafo, nodo_actual, 'Bucharest')
    print(f"Regresando a la Empresa desde {nodo_actual} (Ruta: {camino})")
    distancia_total += distancia

    # Ajustar el fitness con las prioridades
    fitness = distancia_total - alpha * suma_prioridades
    return fitness

# Generar población inicial
def generar_poblacion(paquetes, tamano_poblacion):
    return [
        {'carga': random.sample(paquetes, len(paquetes))}
        for _ in range(tamano_poblacion)
    ]

def normalizar_fitness(poblacion, grafo, capacidad):
    fitness_vals = np.array([calcular_costo_total(ind, grafo, capacidad) for ind in poblacion])
    min_fitness = abs(fitness_vals.min()) + 1  # Asegura que todos los valores sean positivos
    fitness_probs = 1 / (fitness_vals + min_fitness)  # Invertir para que menor costo sea mayor probabilidad
    return fitness_probs / fitness_probs.sum()


def seleccion_por_probabilidad(poblacion, grafo, capacidad):
    fitness_probs = normalizar_fitness(poblacion, grafo, capacidad)
    indices = np.random.choice(len(poblacion), size=2, p=fitness_probs)
    return poblacion[indices[0]], poblacion[indices[1]]



# Cruce
def cruce(padre1, padre2):
    punto_corte = random.randint(1, len(padre1['carga']) - 1)
    hijo1 = {'carga': padre1['carga'][:punto_corte] + padre2['carga'][punto_corte:]}
    hijo2 = {'carga': padre2['carga'][:punto_corte] + padre1['carga'][punto_corte:]}
    return hijo1, hijo2

# Mutación
def mutacion(individuo):
    if random.random() < 0.1:  # Probabilidad de mutación
        idx1, idx2 = random.sample(range(len(individuo['carga'])), 2)
        individuo['carga'][idx1], individuo['carga'][idx2] = individuo['carga'][idx2], individuo['carga'][idx1]


# Algoritmo genético
def algoritmo_genetico(paquetes, grafo, capacidad, generaciones, tamano_poblacion):
    poblacion = generar_poblacion(paquetes, tamano_poblacion)
    for gen in range(generaciones):
        print(f"\n=== Generación {gen + 1} ===")
        print("Individuos y sus fitness:")
        
        # Evaluar e imprimir cada individuo y su fitness
        for i, individuo in enumerate(poblacion):
            fitness = calcular_costo_total(individuo, grafo, capacidad)
            print(f"Individuo {i + 1}: {individuo['carga']}, Fitness: {fitness:.2f}")
        
        # Crear nueva población
        nueva_poblacion = []
        for _ in range(tamano_poblacion // 2):
            padre1, padre2 = seleccion_por_probabilidad(poblacion, grafo, capacidad)
            hijo1, hijo2 = cruce(padre1, padre2)
            mutacion(hijo1)
            mutacion(hijo2)
            nueva_poblacion.extend([hijo1, hijo2])
        
        poblacion = nueva_poblacion

        # Evaluar el mejor de esta generación
        mejor = min(poblacion, key=lambda ind: calcular_costo_total(ind, grafo, capacidad))
        mejor_fitness = calcular_costo_total(mejor, grafo, capacidad)
        print(f"Mejor individuo de la generación: {mejor['carga']}, Fitness: {mejor_fitness:.2f}")

    # Seleccionar el mejor de la última población
    mejor_solucion = min(poblacion, key=lambda ind: calcular_costo_total(ind, grafo, capacidad))
    mejor_fitness = calcular_costo_total(mejor_solucion, grafo, capacidad)
    print(f"\n=== Mejor solución encontrada ===")
    print(f"Secuencia de paquetes: {mejor_solucion['carga']}")
    print(f"Fitness: {mejor_fitness:.2f}")
    return mejor_solucion


# Ejecutar algoritmo
nodos = list(grafo.keys())
paquetes = generar_paquetes(10, [nodo for nodo in nodos if nodo != 'Bucharest'])
capacidad_repartidor = 20

print("Paquetes generados:")
for paquete in paquetes:
    print(paquete)

mejor_solucion = algoritmo_genetico(paquetes, grafo, capacidad_repartidor, generaciones=10, tamano_poblacion=10)
print("\nMejor solución encontrada:", mejor_solucion)


Paquetes generados:
{'peso': 4, 'prioridad': 1, 'nodo_entrega': 'Arad'}
{'peso': 4, 'prioridad': 2, 'nodo_entrega': 'Giurgiu'}
{'peso': 5, 'prioridad': 2, 'nodo_entrega': 'Urziceni'}
{'peso': 4, 'prioridad': 7, 'nodo_entrega': 'Pitesti'}
{'peso': 4, 'prioridad': 10, 'nodo_entrega': 'Vaslui'}
{'peso': 8, 'prioridad': 2, 'nodo_entrega': 'Mehadia'}
{'peso': 4, 'prioridad': 4, 'nodo_entrega': 'Eforie'}
{'peso': 6, 'prioridad': 2, 'nodo_entrega': 'Iasi'}
{'peso': 8, 'prioridad': 8, 'nodo_entrega': 'Giurgiu'}
{'peso': 8, 'prioridad': 1, 'nodo_entrega': 'Oradea'}

=== Generación 1 ===
Individuos y sus fitness:

Evaluando individuo con carga: [{'peso': 6, 'prioridad': 2, 'nodo_entrega': 'Iasi'}, {'peso': 4, 'prioridad': 2, 'nodo_entrega': 'Giurgiu'}, {'peso': 4, 'prioridad': 7, 'nodo_entrega': 'Pitesti'}, {'peso': 8, 'prioridad': 1, 'nodo_entrega': 'Oradea'}, {'peso': 4, 'prioridad': 1, 'nodo_entrega': 'Arad'}, {'peso': 4, 'prioridad': 10, 'nodo_entrega': 'Vaslui'}, {'peso': 8, 'prioridad': 8,